In [1]:
%%configure
{
    "vCores": 64
}

In [2]:
!pip install  duckdb --upgrade
import sys
sys.exit(0)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/20.2 MB 50.1 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: duckdb
    Found existing installation: duckdb 1.1.3
    Uninstalling duckdb-1.1.3:
sys.exit called with value 0. The interpreter will be restarted.
      Successfully uninstalled duckdb-1.1.3


In [3]:
# https://github.com/JosueBogran/coffeeshopdatagenerator/

In [4]:
import duckdb
from   deltalake.writer import write_deltalake
from   datetime import datetime

In [5]:
start = time.time()
print(start)

1742714663.7794962


In [6]:
con = duckdb.connect()
con.sql(" force install delta from core_nightly ; update extensions ; ")

┌────────────────┬──────────────┬─────────────────────┬──────────────────┬─────────────────┐
│ extension_name │  repository  │    update_result    │ previous_version │ current_version │
│    varchar     │   varchar    │       varchar       │     varchar      │     varchar     │
├────────────────┼──────────────┼─────────────────────┼──────────────────┼─────────────────┤
│ delta          │ core_nightly │ NO_UPDATE_AVAILABLE │ 026345b          │ 026345b         │
│ azure          │ core         │ NO_UPDATE_AVAILABLE │ e707cf3          │ e707cf3         │
└────────────────┴──────────────┴─────────────────────┴──────────────────┴─────────────────┘

In [7]:
con.sql(" SET temp_directory='/tmp' ")
con.sql(" create schema if not exists sweetcoffeetree ")
con.sql(" use sweetcoffeetree ")
for tbl in ['dim_locations','dim_products','facts_sales_3b_rows']:
    con.sql(f"""   attach '/lakehouse/default/Tables/sweetcoffeetree/{tbl}' as {tbl} (type delta, pin_snapshot) """)


In [8]:
con.sql(" show all tables ").show(max_width = 120)

┌─────────────────────┬─────────┬─────────────────────┬──────────────────────┬─────────────────────────────┬───────────┐
│      database       │ schema  │        name         │     column_names     │        column_types         │ temporary │
│       varchar       │ varchar │       varchar       │      varchar[]       │          varchar[]          │  boolean  │
├─────────────────────┼─────────┼─────────────────────┼──────────────────────┼─────────────────────────────┼───────────┤
│ dim_locations       │ main    │ dim_locations       │ [record_id, locati…  │ [INTEGER, VARCHAR, VARCHA…  │ false     │
│ dim_products        │ main    │ dim_products        │ [record_id, produc…  │ [INTEGER, INTEGER, VARCHA…  │ false     │
│ facts_sales_3b_rows │ main    │ facts_sales_3b_rows │ [Order_ID, order_l…  │ [VARCHAR, VARCHAR, DATE, …  │ false     │
└─────────────────────┴─────────┴─────────────────────┴──────────────────────┴─────────────────────────────┴───────────┘



In [9]:
%%time
con.sql("""
-- 1) Calculate total daily sales for each city and a 7-day rolling average.
SELECT
    f.order_date,
    l.city,
    SUM(f.sales_amount) AS total_sales,
    AVG(SUM(f.sales_amount)) OVER (
        PARTITION BY l.city
        ORDER BY f.order_date
        ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
    ) AS rolling_7day_avg
FROM facts_sales_3b_rows f
JOIN dim_locations l
    ON f.location_id = l.location_id
GROUP BY
    f.order_date,
    l.city
ORDER BY
    l.city,
    f.order_date;
""").show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌────────────┬─────────────┬────────────────────┬────────────────────┐
│ order_date │    city     │    total_sales     │  rolling_7day_avg  │
│    date    │   varchar   │       double       │       double       │
├────────────┼─────────────┼────────────────────┼────────────────────┤
│ 2023-01-01 │ Austin      │   8820512.05699983 │   8820512.05699983 │
│ 2023-01-02 │ Austin      │  8798169.292499842 │  8809340.674749836 │
│ 2023-01-03 │ Austin      │   8827154.27349984 │  8815278.540999837 │
│ 2023-01-04 │ Austin      │  8803562.383499848 │   8812349.50162484 │
│ 2023-01-05 │ Austin      │  8808153.085499855 │  8811510.218399841 │
│ 2023-01-06 │ Austin      │  8806710.058999848 │  8810710.191833176 │
│ 2023-01-07 │ Austin      │  8812655.754499845 │  8810988.129356986 │
│ 2023-01-08 │ Austin      │   8813754.51999984 │  8810022.766928418 │
│ 2023-01-09 │ Austin      │  8802010.993499847 │   8810571.58135699 │
│ 2023-01-10 │ Austin      │  8811449.454999838 │  8808328.035856988 │
│     

In [10]:
%%time
con.sql("""
-- 2) For each month, rank products by total sales amount, with 1 being the highest.
WITH monthly_sales AS (
    SELECT
        DATE_TRUNC('month', f.order_date) AS sales_month,
        f.product_name,
        SUM(f.sales_amount) AS total_sales
    FROM facts_sales_3b_rows f
    GROUP BY
        DATE_TRUNC('month', f.order_date),
        f.product_name
)
SELECT
    sales_month,
    product_name,
    total_sales,
    RANK() OVER (PARTITION BY sales_month ORDER BY total_sales DESC) AS sales_rank
FROM monthly_sales
ORDER BY sales_month, sales_rank;
""").show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌─────────────┬────────────────────────┬────────────────────┬────────────┐
│ sales_month │      product_name      │    total_sales     │ sales_rank │
│    date     │        varchar         │       double       │   int64    │
├─────────────┼────────────────────────┼────────────────────┼────────────┤
│ 2023-01-01  │ Cappuccino             │ 119641763.45796834 │          1 │
│ 2023-01-01  │ Latte                  │ 109860638.44499044 │          2 │
│ 2023-01-01  │ Americano              │ 107426181.01594543 │          3 │
│ 2023-01-01  │ Macchiato              │   97661777.7199992 │          4 │
│ 2023-01-01  │ Chamomile              │  85450917.51000248 │          5 │
│ 2023-01-01  │ Frappe                 │  82220159.47500852 │          6 │
│ 2023-01-01  │ Chai                   │  78158699.42393193 │          7 │
│ 2023-01-01  │ Iced Latte             │  76788539.35500214 │          8 │
│ 2023-01-01  │ Green                  │  75683799.99295454 │          9 │
│ 2023-01-01  │ Pumpkin B

In [11]:
%%time
con.sql("""
-- 3) Find the locations in each season with the highest average discount, limited to top 3.
WITH season_discount AS (
    SELECT
        l.city,
        l.state,
        f.season,
        AVG(f.discount_percentage) AS avg_discount
    FROM facts_sales_3b_rows f
    JOIN dim_locations l
        ON f.location_id = l.location_id
    GROUP BY
        l.city,
        l.state,
        f.season
)
SELECT
    city,
    state,
    season,
    avg_discount,
    discount_rank
FROM (
    SELECT
        city,
        state,
        season,
        avg_discount,
        DENSE_RANK() OVER (PARTITION BY season ORDER BY avg_discount DESC) AS discount_rank
    FROM season_discount
) t
WHERE discount_rank <= 3
ORDER BY season, discount_rank;
""").show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌─────────────┬─────────┬─────────┬────────────────────┬───────────────┐
│    city     │  state  │ season  │    avg_discount    │ discount_rank │
│   varchar   │ varchar │ varchar │       double       │     int64     │
├─────────────┼─────────┼─────────┼────────────────────┼───────────────┤
│ Austin      │ TX      │ fall    │  1.600513619952045 │             1 │
│ Los Angeles │ CA      │ fall    │ 1.6003507089908546 │             2 │
│ Charlotte   │ NC      │ fall    │ 1.5999903715352268 │             3 │
│ Charlotte   │ NC      │ spring  │ 1.6001685568161548 │             1 │
│ Los Angeles │ CA      │ spring  │ 1.6001466328124614 │             2 │
│ Austin      │ TX      │ spring  │  1.599839272249681 │             3 │
│ Charlotte   │ NC      │ summer  │  1.600294025512906 │             1 │
│ Houston     │ TX      │ summer  │ 1.6001545386906468 │             2 │
│ Austin      │ TX      │ summer  │ 1.5999526048166153 │             3 │
│ Austin      │ TX      │ winter  │ 1.6004225253056

In [12]:
%%time
con.sql("""
-- 4) Compare actual daily sales to standard_price and standard_cost, to show total margin.
--    Join on product_name and date range.
SELECT
    f.order_date,
    f.product_name,
    p.standard_price,
    p.standard_cost,
    SUM(f.quantity) AS total_quantity_sold,
    SUM(f.sales_amount) AS total_sales_amount,
    (p.standard_price - p.standard_cost) * SUM(f.quantity) AS theoretical_margin
FROM facts_sales_3b_rows f
JOIN dim_products p
    ON f.product_name = p.name
    AND f.order_date BETWEEN p.from_date AND p.to_date
GROUP BY
    f.order_date,
    f.product_name,
    p.standard_price,
    p.standard_cost
ORDER BY
    f.order_date,
    f.product_name;
""").show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌────────────┬────────────────────────┬────────────────┬───────────────┬─────────────────────┬────────────────────┬────────────────────┐
│ order_date │      product_name      │ standard_price │ standard_cost │ total_quantity_sold │ total_sales_amount │ theoretical_margin │
│    date    │        varchar         │     double     │    double     │       int128        │       double       │       double       │
├────────────┼────────────────────────┼────────────────┼───────────────┼─────────────────────┼────────────────────┼────────────────────┤
│ 2023-01-01 │ Americano              │            4.4 │           1.5 │              799856 │ 3463477.9960001195 │ 2319582.4000000004 │
│ 2023-01-01 │ Biscotti               │            3.9 │           1.2 │              611150 │   2345361.83700005 │          1650105.0 │
│ 2023-01-01 │ Black                  │            3.0 │           0.3 │              801321 │ 2365412.6399999964 │          2163566.7 │
│ 2023-01-01 │ Cappuccino             │  

In [13]:
%%time
con.sql("""
-- 5) Use a window function to calculate a 30-day rolling total quantity sold per city.
WITH daily_city_qty AS (
    SELECT
        f.order_date,
        l.city,
        SUM(f.quantity) AS daily_qty
    FROM facts_sales_3b_rows f
    JOIN dim_locations l
        ON f.location_id = l.location_id
    GROUP BY
        f.order_date,
        l.city
)
SELECT
    order_date,
    city,
    daily_qty,
    SUM(daily_qty) OVER (
        PARTITION BY city
        ORDER BY order_date
        ROWS BETWEEN 29 PRECEDING AND CURRENT ROW
    ) AS rolling_30day_qty
FROM daily_city_qty
ORDER BY city, order_date;
""").show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌────────────┬─────────────┬───────────┬───────────────────┐
│ order_date │    city     │ daily_qty │ rolling_30day_qty │
│    date    │   varchar   │  int128   │      int128       │
├────────────┼─────────────┼───────────┼───────────────────┤
│ 2023-01-01 │ Austin      │   2245688 │           2245688 │
│ 2023-01-02 │ Austin      │   2241289 │           4486977 │
│ 2023-01-03 │ Austin      │   2247586 │           6734563 │
│ 2023-01-04 │ Austin      │   2241804 │           8976367 │
│ 2023-01-05 │ Austin      │   2242654 │          11219021 │
│ 2023-01-06 │ Austin      │   2242521 │          13461542 │
│ 2023-01-07 │ Austin      │   2244559 │          15706101 │
│ 2023-01-08 │ Austin      │   2244302 │          17950403 │
│ 2023-01-09 │ Austin      │   2242403 │          20192806 │
│ 2023-01-10 │ Austin      │   2244084 │          22436890 │
│     ·      │   ·         │      ·    │              ·    │
│     ·      │   ·         │      ·    │              ·    │
│     ·      │   ·      

In [14]:
%%time
df = con.sql("""
-- 6) Create or replace a table that stores monthly revenue by product category.
WITH monthly_cat AS (
    SELECT
        DATE_TRUNC('month', f.order_date) AS sales_month,
        p.category,
        SUM(f.sales_amount) AS monthly_revenue
    FROM facts_sales_3b_rows f
    JOIN dim_products p
        ON f.product_name = p.name
        AND f.order_date BETWEEN p.from_date AND p.to_date
    GROUP BY
        DATE_TRUNC('month', f.order_date),
        p.category
)
SELECT
    sales_month,
    category,
    monthly_revenue
FROM monthly_cat;
""").record_batch()
write_deltalake(f"abfss://sqlengines@onelake.dfs.fabric.microsoft.com/coffee.Lakehouse/Tables/sweetcoffeetree/monthly",
                 df, mode="overwrite")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 9min 13s, sys: 37.2 s, total: 9min 51s
Wall time: 46.7 s


In [15]:
%%time
con.sql("""
-- 7) Compare total sales by location in 2023 vs. 2024.
WITH yearly_sales AS (
    SELECT
        l.location_id,
        l.city,
        l.state,
        YEAR(f.order_date) AS sales_year,
        SUM(f.sales_amount) AS total_sales_year
    FROM facts_sales_3b_rows f
    JOIN dim_locations l
        ON f.location_id = l.location_id
    GROUP BY
        l.location_id,
        l.city,
        l.state,
        YEAR(f.order_date)
)
SELECT
    city,
    state,
    SUM(CASE WHEN sales_year = 2023 THEN total_sales_year ELSE 0 END) AS sales_2023,
    SUM(CASE WHEN sales_year = 2024 THEN total_sales_year ELSE 0 END) AS sales_2024,
    (SUM(CASE WHEN sales_year = 2024 THEN total_sales_year ELSE 0 END)
     - SUM(CASE WHEN sales_year = 2023 THEN total_sales_year ELSE 0 END)) AS yoy_diff
FROM yearly_sales
GROUP BY
    city,
    state
ORDER BY
    city,
    state;
""").show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌─────────────┬─────────┬────────────────────┬────────────────────┬───────────────────┐
│    city     │  state  │     sales_2023     │     sales_2024     │     yoy_diff      │
│   varchar   │ varchar │       double       │       double       │      double       │
├─────────────┼─────────┼────────────────────┼────────────────────┼───────────────────┤
│ Austin      │ TX      │ 3797920063.8126516 │   3889741574.63715 │ 91821510.82449818 │
│ Charlotte   │ NC      │  3849584167.163656 │ 3943153782.8071556 │ 93569615.64349937 │
│ Houston     │ TX      │ 3891261321.1591454 │ 3985247069.5641413 │ 93985748.40499592 │
│ Los Angeles │ CA      │ 3849566860.1296563 │  3943415476.759157 │ 93848616.62950087 │
└─────────────┴─────────┴────────────────────┴────────────────────┴───────────────────┘

CPU times: user 9min 48s, sys: 28.9 s, total: 10min 17s
Wall time: 38.5 s


In [16]:
%%time
con.sql("""
-- 8) For each city and quarter, rank subcategories by total sales amount.
WITH city_quarter_subcat AS (
    SELECT
        l.city,
        DATE_TRUNC('quarter', f.order_date) AS sales_quarter,
        p.subcategory,
        SUM(f.sales_amount) AS total_sales
    FROM facts_sales_3b_rows f
    JOIN dim_locations l
        ON f.location_id = l.location_id
    JOIN dim_products p
        ON f.product_name = p.name
        AND f.order_date BETWEEN p.from_date AND p.to_date
    GROUP BY
        l.city,
        DATE_TRUNC('quarter', f.order_date),
        p.subcategory
)
SELECT
    city,
    sales_quarter,
    subcategory,
    total_sales,
    RANK() OVER (PARTITION BY city, sales_quarter ORDER BY total_sales DESC) AS subcat_rank
FROM city_quarter_subcat
ORDER BY city, sales_quarter, subcat_rank;
""").show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌─────────────┬───────────────┬─────────────┬────────────────────┬─────────────┐
│    city     │ sales_quarter │ subcategory │    total_sales     │ subcat_rank │
│   varchar   │     date      │   varchar   │       double       │    int64    │
├─────────────┼───────────────┼─────────────┼────────────────────┼─────────────┤
│ Austin      │ 2023-01-01    │ Coffee      │ 445379799.40334344 │           1 │
│ Austin      │ 2023-01-01    │ Tea         │  240015755.9710379 │           2 │
│ Austin      │ 2023-01-01    │ Pastries    │  156252309.9294607 │           3 │
│ Austin      │ 2023-04-01    │ Coffee      │   582162331.396179 │           1 │
│ Austin      │ 2023-04-01    │ Tea         │  250141286.2816877 │           2 │
│ Austin      │ 2023-04-01    │ Pastries    │ 145422256.22201756 │           3 │
│ Austin      │ 2023-07-01    │ Coffee      │  682126204.9588834 │           1 │
│ Austin      │ 2023-07-01    │ Tea         │ 225373187.42469433 │           2 │
│ Austin      │ 2023-07-01  

In [17]:
%%time
con.sql("""
-- 9) Show average discount by day, and a running cumulative average discount per city.
WITH daily_discount AS (
    SELECT
        l.city,
        f.order_date,
        AVG(f.discount_percentage) AS avg_discount
    FROM facts_sales_3b_rows f
    JOIN dim_locations l
        ON f.location_id = l.location_id
    GROUP BY
        l.city,
        f.order_date
)
SELECT
    city,
    order_date,
    avg_discount,
    AVG(avg_discount) OVER (
        PARTITION BY city
        ORDER BY order_date
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) AS cumulative_avg_discount
FROM daily_discount
ORDER BY city, order_date;
""").show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌─────────────┬────────────┬────────────────────┬─────────────────────────┐
│    city     │ order_date │    avg_discount    │ cumulative_avg_discount │
│   varchar   │    date    │       double       │         double          │
├─────────────┼────────────┼────────────────────┼─────────────────────────┤
│ Austin      │ 2023-01-01 │ 1.5994611715264406 │      1.5994611715264406 │
│ Austin      │ 2023-01-02 │ 1.6008006927341634 │      1.6001309321303019 │
│ Austin      │ 2023-01-03 │  1.606974773348831 │      1.6024122125364784 │
│ Austin      │ 2023-01-04 │ 1.6024533098614628 │      1.6024224868677246 │
│ Austin      │ 2023-01-05 │  1.595708385958769 │      1.6010796666859335 │
│ Austin      │ 2023-01-06 │ 1.5951350789322578 │      1.6000889020603208 │
│ Austin      │ 2023-01-07 │ 1.5974135139031116 │       1.599706703752148 │
│ Austin      │ 2023-01-08 │ 1.6093388569670088 │      1.6009107229040056 │
│ Austin      │ 2023-01-09 │ 1.6002030919056447 │       1.600832097237521 │
│ Austin    

In [18]:
%%time
df = con.sql("""
-- 10) Create/replace table for 90-day rolling count of distinct orders in each city.
WITH daily_orders as (
    SELECT
        f.order_date,
        l.city,
        COUNT(DISTINCT f.order_id) AS daily_distinct_orders
    FROM facts_sales_3b_rows f
    JOIN dim_locations l
        ON f.location_id = l.location_id
    GROUP BY
        f.order_date,
        l.city
)
SELECT
    order_date,
    city,
    daily_distinct_orders,
    SUM(daily_distinct_orders) OVER (
        PARTITION BY city
        ORDER BY order_date
        ROWS BETWEEN 89 PRECEDING AND CURRENT ROW
    ) AS rolling_90d_distinct_orders
FROM daily_orders
ORDER BY city, order_date;
""").record_batch()
write_deltalake(f"abfss://sqlengines@onelake.dfs.fabric.microsoft.com/coffee.Lakehouse/Tables/sweetcoffeetree/distinctorders",
              df, mode="overwrite")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 55min 5s, sys: 17min 30s, total: 1h 12min 36s
Wall time: 3min 1s


In [19]:
print(time.time()-start)

632.0109188556671
